<a href="https://colab.research.google.com/github/YixinFan11/DIA-Submission/blob/master/WhatsApp_Chat_Summary_Final(DIA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WhatsApp Chat Summary

## Data Preparation

In [1]:
## The WhatsApp chat history has been loaded on google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
## Install pacages
!pip install transformers
!pip install bert-extractive-summarizer
!pip install pytorch-transformers
!pip install transformers torch sentencepiece
!pip install rouge_score

# !pip install streamlit
# !pip install stqdm

     |████████████████████████████████| 4.2 MB 4.9 MB/s 
     |████████████████████████████████| 6.6 MB 43.1 MB/s 
     |████████████████████████████████| 84 kB 2.9 MB/s 
     |████████████████████████████████| 596 kB 69.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 176 kB 5.0 MB/s 
     |████████████████████████████████| 1.2 MB 54.8 MB/s 
     |████████████████████████████████| 880 kB 59.7 MB/s 
     |████████████████████████████████| 132 kB 66.0 MB/s 
     |████████████████████████████████| 8.7 MB 51.0 MB/s 
     |████████████████████████████████| 79 kB 7.0 MB/s 
     |████████████████████████████████| 138 kB 70.7 MB/s 
     |████████████████████████████████| 127 kB 73.7 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=8cf60cc803ca579a2e174e1bf1ac8250b49fa812ba3a3b15ecef8e02989f2e53
 

In [3]:
# Import relative packages for data preparation
## Data preparation
import numpy as np
import re
import datetime 

## Model construction
from transformers import pipeline
from tokenizers import Tokenizer
from summarizer import Summarizer,TransformerSummarizer
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

## Evaluation
from rouge_score import rouge_scorer


In [4]:
## Read the file that saved in google drive
with open('drive/MyDrive/ChatHistory.txt', 'r') as file:
    data = file.read()

In [5]:
## Delete the 
def preprocess_chats(text):
    # Make everything in lower case
    text = text.lower()

    # Refer - https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python 
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

    # chars_and_links= r'\d+|http?\S+|[^A-Za-z0-9]+'
    # text = re.sub(chars_and_links,' ',text)
    text = re.sub(emoji_pattern,' ',text)

    patterns = [r"\d{2}:\d{2}\s*",    #sun aug 19 13:02:10 2018
        r":\s*([\da-zA_Z0_9]+\/)+([a-zA-Z0-9\.]+)",                #URL
        r"([a-zA-Z_!]+)[\.!_]\d+:\s*",                          #word[._!]number:>=0space
        r":\d+",
        "[':,${}\[\].–|%\"=]"]

    for p in patterns:
        text = re.sub(p,'', text)

    text = text.strip()
    return text


In [6]:
# 
bad_words = ['media omitted', 'this message was deleted','deleted this message','http','xd','damn','fuck','lol','anshu']   

def remove(bad_words_list,text):
    my_string=preprocess_chats(text)
    for words in bad_words_list:
        my_string=re.sub(".*"+words+".*"+"\n","",my_string)    
    return my_string


In [7]:

def validate(date_text):
    try:
        if date_text != datetime.datetime.strptime(date_text, "%d/%m/%Y").strftime('%d/%m/%Y'):
            raise ValueError
        return True
    except ValueError:
        return False


In [8]:
def date_wise_history(bad_words_list,text):
    text=remove(bad_words,text)
    chat_history = {} # Intiallize the dict.    
    split_text=text.split("\n")
    for lines in split_text:
        new_line=lines.split(" -")
        if validate(new_line[0])==True:
            if new_line[0] in chat_history.keys():
                chat_history[new_line[0]]+=new_line[1:len(new_line)+1]
            else:
                chat_history[new_line[0]]=new_line[1:len(new_line)+1]
        else:
            ind=split_text.index(lines)
            lines = re.sub('\t', '', lines)
            relative_chat=split_text[ind-1]
            date=relative_chat.split(' -')[0]
            chat_history[date].append(lines)
            split_text[ind]=date+' - '+lines
    return chat_history

In [9]:
user_list = ['yixin', 'jackson']
def date_selection_chat(bad_words,user_list,start_dat, end_dat,text):
    start = datetime.datetime.strptime(start_dat, "%d/%m/%Y")
    end = datetime.datetime.strptime(end_dat, "%d/%m/%Y")
    date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days+1)]
    date_lst=list(map(lambda x: x.strftime("%d/%m/%Y"),date_generated))
    dic=date_wise_history(bad_words,text)
    key_list=list(dic.keys())
    first_date = datetime.datetime.strptime(key_list[0], "%d/%m/%Y")
    last_date = datetime.datetime.strptime(key_list[-1], "%d/%m/%Y")
    if start>end:
        select_chat=False
    elif last_date<start:
        select_chat=False
    elif first_date>end:
        select_chat=False
        #return False
    else:
        select_chat={}
        for key,values in dic.items():
            if key in date_lst:
                chats_of_the_day = ''
                for chat in values : 
                    chats_of_the_day += chat;
                select_chat[key]=chats_of_the_day
    select_chat_1={}
    select_chat_2={}
    for key,value in select_chat.items():
        value_1=value
        value_2=value
        for user in user_list:
            value_1=re.sub(user," ", value_1)
            value_2=re.sub(user,user+":", value_2)
        select_chat_1[key]=value_1
        select_chat_2[key]=value_2
    return select_chat_1,select_chat_2


## Create Summary

In [10]:
def bert_summary(bad_words,user_list,start_dat, end_dat,text):
    select_chat_dic,select_chat_dic_user=date_selection_chat(bad_words,user_list,start_dat, end_dat,text)
    if select_chat_dic == False:
        print('Current Date does not match your chat history, please enter a valid start date')
    elif bool(select_chat_dic)==False:
        print("No summary available")
    else:
        chat_sum_dict_bert = {}
        for key,value in select_chat_dic.items():   
            bert_model = Summarizer()
            bert_summary = ''.join(bert_model(value, min_length=50,max_length=150))
            chat_sum_dict_bert[key] = str(bert_summary)
        return chat_sum_dict_bert

In [11]:
def gpt2_summary(bad_words,user_list,start_dat, end_dat,text):
    select_chat_dic,select_chat_dic_user=date_selection_chat(bad_words,user_list,start_dat, end_dat,text)
    if select_chat_dic == False:
        print('Current Date does not match your chat history, please enter a valid start date')
    elif bool(select_chat_dic)==False:
        print("No summary available")
    else:
        chat_sum_dict_gpt2 = {}
        for key,value in select_chat_dic.items():
            GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")
            GPT2_summary = ''.join(GPT2_model(value, min_length=60))
            chat_sum_dict_gpt2[key] = str(GPT2_summary)
        return chat_sum_dict_gpt2

In [12]:
def distilbart_summary(bad_words,user_list,start_dat, end_dat,text):
    select_chat_dic,select_chat_dic_user=date_selection_chat(bad_words,user_list,start_dat, end_dat,text)
    if select_chat_dic_user == False:
        print('Current Date does not match your chat history, please enter a valid start date')
    elif bool(select_chat_dic_user)==False:
        print("No summary available")
    else:
        chat_sum_dict_distilbart = {}
        for key,value in select_chat_dic_user.items():
            summarization = pipeline("summarization",truncation=True, min_length = 5, max_length = 80,model="philschmid/distilbart-cnn-12-6-samsum")
            summary_text = summarization(value)[0]['summary_text']
            chat_sum_dict_distilbart[key] = str(summary_text)
        return chat_sum_dict_distilbart


In [25]:

def pegasus_summary(bad_words,user_list,start_dat, end_dat,text):
    select_chat_dic,select_chat_dic_user=date_selection_chat(bad_words,user_list,start_dat, end_dat,text)
    if select_chat_dic == False:
        print('Current Date does not match your chat history, please enter a valid start date')
    elif bool(select_chat_dic)==False:
        print("No summary available")
    else:
        chat_sum_dict_pegasus = {}
        model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
        tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")
        for key,value in select_chat_dic.items():
            inputs = tokenizer(value, truncation=True, max_length=120,padding="longest", return_tensors="pt")
            summary_ids = model.generate(inputs["input_ids"])
            summary_text = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
            chat_sum_dict_pegasus[key] = str(summary_text)
        return chat_sum_dict_pegasus

In [14]:
original_data_1,original_data_2=date_selection_chat(bad_words,user_list,'15/03/2022', '25/03/2022',data)

In [15]:
bert=bert_summary(bad_words,user_list,'15/03/2022', '25/03/2022',data)

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.

In [16]:
gpt2=gpt2_summary(bad_words,user_list,'15/03/2022', '25/03/2022',data)

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

In [17]:
distilbart=distilbart_summary(bad_words,user_list,'15/03/2022', '25/03/2022',data)

Downloading:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [26]:
pegasus=pegasus_summary(bad_words,user_list,'15/03/2022', '25/03/2022',data)

In [19]:
original_data_1

{'15/03/2022': '   am now fighting with the bank   they deliver my new card and i have no idea where they deliver   nice blog\u3000ありがとうございます   which bank do you have account in? barclay?   yep   oh well it will reach to you eventually so dont worry   am just afraid he deliver to myprevious address   in sussex ?   kind of   not sure   i just wanna check   i got my ni number   you can visit the main branch in nottingham tomorrow   nice   nice   you got it pretty late ?   i got it last to last week   not sure   i just not check my mail box   okay   so what is the next step ?   finished big data   now?   after i call barclay   in 20min i hope   am looking at bogdata   when i chat with them just now   yeah no worries you can finish the thing first with the barclay   if you need to go to the main branch i can join you on thursday after that i come to your place and we discuss about the mlis project and other things?   the new project released   the project is nice   but the supervisor is sh

In [20]:
original_data_2

{'15/03/2022': ' yixin: am now fighting with the bank yixin: they deliver my new card and i have no idea where they deliver yixin: nice blog\u3000ありがとうございます jackson: which bank do you have account in? barclay? yixin: yep jackson: oh well it will reach to you eventually so dont worry yixin: am just afraid he deliver to myprevious address jackson: in sussex ? yixin: kind of yixin: not sure yixin: i just wanna check yixin: i got my ni number jackson: you can visit the main branch in nottingham tomorrow yixin: nice jackson: nice jackson: you got it pretty late ? jackson: i got it last to last week yixin: not sure yixin: i just not check my mail box jackson: okay jackson: so what is the next step ? jackson: finished big data jackson: now? yixin: after i call barclay yixin: in 20min i hope yixin: am looking at bogdata yixin: when i chat with them just now jackson: yeah no worries you can finish the thing first with the barclay jackson: if you need to go to the main branch i can join you on t

In [21]:
bert

{'15/03/2022': 'yep   oh well it will reach to you eventually so dont worry   am just afraid he deliver to myprevious address   in sussex ? hahhhh   yeah he is like that   so did you think of 3 projects ?',
 '16/03/2022': 'and i have not yet finish the project   it took 90 mins? am free tomorrow but not convenient   yup no worries   friday is fine   i will go for lecture and we candiscuss then   friday or tomorrow?',
 '18/03/2022': '',
 '19/03/2022': 'i was just working on mlis project   it is easy 3 days of coding     and done   i remember you are wasting your time today？ big data   okay   and gave a rough text of our submission   you said and i wrote   kind of   and show the result to dalila and div   ？',
 '20/03/2022': 'am thinking of buying a new mac   yup i am here   what have you thought about ?',
 '21/03/2022': 'yeah   niceask for verbal feedback then   yeah they give us that   can you see your marks ? i didnt see   but i dont think we can see that   i havent check     just chec

In [22]:
gpt2

{'15/03/2022': 'am now fighting with the bank   they deliver my new card and i have no idea where they deliver   nice blog\u3000ありがとうございます   which bank do you have account in? yep   oh well it will reach to you eventually so dont worry   am just afraid he deliver to myprevious address   in sussex ? 20406477   all of three of us has to send this email right   i think so   you added us in cc   nice   people have started training the mlis code   we also have to try that   we have to start righting the code   you first   i will start to do it after big data   wait i will call are you free   dc   okay   d   didnt notice it   allocation ?',
 '16/03/2022': 'i may miss sth it contains toooooooo much information   yeah i can see highlights   it took me nearly 90 mins to read it   i will mark my highlights with different color   what ? i dont think   dk，just see if i can help   i will ask him   but i dont think the last i saw him was 2 week ago   could buy some medicine for this   when we meet a

In [23]:
distilbart

{'15/03/2022': " yixin's new card was delivered to her new address. Jackson got it late, but he got it last week. Yixin and Jackson will visit the main branch in nottingham tomorrow. They will discuss the mlis project and other things on Thursday. ",
 '16/03/2022': " yixin and Jackson can't start the big data project. They need to do 3 pre and 4 reports by tomorrow. They have to do the express of interest, the project interest and the project detail. They also need to apply for their jobs. They are meeting tomorrow in the library to discuss the project.",
 '18/03/2022': 'Yixin has reached home. She is on a call. Jackson and Dalila will do a group call in an hour or so. ',
 '19/03/2022': 'Yixin and Jackson are working on a project on big data. They will discuss it tomorrow afternoon. ',
 '20/03/2022': ' yixin is thinking of buying a new mac. The fans are loud and hot and the mac gets stuck when running data bricks. Jackson and Yixin will discuss problems with the mac tomorrow after the 

In [24]:
pegasus

{'15/03/2022': "I got my new bank card last week but i don't know how to use it.",
 '16/03/2022': 'Today is the last day to do the bigdata we need to do 3 pre and 4 reports welcome to my world we need to do 3 pre and 4 reports welcome to my world we need to do 3 pre and 4 reports welcome to my world we need to do 3 pre and 4 reports welcome to my world',
 '18/03/2022': '.',
 '19/03/2022': 'sorry for the mess i created yesterday this sentence had one line missing that the nicest thing someone said to today was that i was a important person to them hahhh my leg is sour too much walk yesterday are you fine ?',
 '20/03/2022': 'we are going to talk about the big data and how to use it in class today we are going to talk about the big data and how to use it in class today we are going to talk about the big data and how to use it in class today we are going to talk about the big data and how',
 '21/03/2022': "hi gauss i'm sitting in class and i'm feeling sick and i'm not sure if i should carr

## Evaluation(Rouge Score)

In [ ]:
# 2022/03/25
baseline=str(['yixin miss the blood test due to the stomachache. jackson got a new bank card. jackson suggest to go to the bank monday after lecture. yixin recieved a second new bank card. yixin do not like the Chinese food ordered yesterday because it cause the stomachache'])


summary_bert=str(bert['25/03/2022'])
summary_gpt2=str(gpt2['25/03/2022'])
summary_distibart=str(distilbart['25/03/2022'])
summary_pegasus=str(pegasus['25/03/2022'])

def calc_rouge_scores(summary, baseline):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    scores = scorer.score(baseline,summary)
    return scores


In [ ]:
rouge_bert = calc_rouge_scores(summary_bert, baseline)
rouge_bert

{'rouge1': Score(precision=0.14705882352941177, recall=0.10869565217391304, fmeasure=0.125),
 'rougeL': Score(precision=0.11764705882352941, recall=0.08695652173913043, fmeasure=0.09999999999999999)}

In [ ]:
rouge_gpt2 = calc_rouge_scores(summary_gpt2, baseline)
rouge_gpt2

{'rouge1': Score(precision=0.23943661971830985, recall=0.3695652173913043, fmeasure=0.29059829059829057),
 'rougeL': Score(precision=0.16901408450704225, recall=0.2608695652173913, fmeasure=0.20512820512820512)}

In [ ]:
rouge_distibart = calc_rouge_scores(summary_distibart, baseline)
rouge_distibart

{'rouge1': Score(precision=0.5555555555555556, recall=0.32608695652173914, fmeasure=0.410958904109589),
 'rougeL': Score(precision=0.4074074074074074, recall=0.2391304347826087, fmeasure=0.30136986301369867)}

In [ ]:
rouge_pegasus = calc_rouge_scores(summary_pegasus, baseline)
rouge_pegasus

{'rouge1': Score(precision=0.5, recall=0.08695652173913043, fmeasure=0.14814814814814817),
 'rougeL': Score(precision=0.5, recall=0.08695652173913043, fmeasure=0.14814814814814817)}